# GMMHMM

In [1]:
# mixture model class
class GMMInfo:
    def __init__(self):
        self.weight=[] #gmm weight
        self.mean=[] #gmm mean
        self.var=[] # gmm diagonal covariance
        self.num=0 # number of gmms
# hmm class
class HMMInfo:
    def __init__(self):
        self.init=[]
        self.edge_cost=[]
        self.mix=[]
        self.num=0

In [2]:
def log_Gaussian(m,C,x):
    C=np.array(C)
    left=0.5*np.sum(np.log((2*np.pi)*C))
    right=0.5*np.sum(np.square((x-m))/C)
    return left+right

def mixture_log_Gaussian(mix,x):
    mu=mix.mean
    var=mix.var
    w=mix.weight
    cost=log_Gaussian(mu,var,x)
    # print(w,cost)
    w=np.array(w)
    total_cost=np.sum(w*cost)
    return total_cost

def Gaussian(m,C,x):
    C=np.array(C)
    part1=np.sqrt((2*np.pi)*C)
    part2=0.5*np.sum(np.square(x-m)/C)
    prob=(1/part1)*np.exp(-part2)
    return prob


def mix_Gaussian(mix,x):
    total_prob=0.0
    for i in range(mix.num):
        m=mix.mean[i][:]
        var=mix.var[i][:]
        w=mix.weight[i]
        prob=Gaussian(m,var,x)
        total_prob+=w*prob
    return total_prob

In [3]:
import numpy as np
def GMMHMM_DTW(HMM,data):
    # matrix that records the edge costs
    T=HMM.edge_cost
    zeros=np.zeros([39])
    ones=np.zeros([39])+1
    mixture_models=[]
    #create a GMM for the initial state
    init_GMM=GMMInfo()
    init_GMM.weight=[1]
    init_GMM.num=1
    init_GMM.mean.append(zeros)
    init_GMM.var.append(ones)
    mixture_models.append(init_GMM)
    for mix_model in HMM.mix:
        mixture_models.append(mix_model)
    data=np.vstack([zeros,data])
    s=len(data)
    t=len(mixture_models)
    #Matrix that stores the costs
    P=np.zeros([t,s])
    #dynamic time programming algo
    for j in range(0,s):
        for i in range(t):
            #node score
            # print((i,j))
            Cij=mixture_log_Gaussian(mixture_models[i],data[j])
            
                
            if i>=2:
                P[i][j]=min(P[i][j-1]+T[i][i],P[i-1][j-1]+T[i-1][i],
                            P[i-2][j-1]+T[i-2][i])+Cij
            elif i-1>=0:
                P[i][j]=min(P[i][j-1]+T[i][i],P[i-1][j-1]+T[i-1][i])+Cij
            else:
                P[i][j]=P[i][j]+Cij
    P=P/s
    total_cost=P[-1][-1]
    return total_cost,get_states(P)

In [4]:
# Get the list that records which state each frame belongs to
def get_states(P):
    current_state,current_frame=np.array(P.shape)-1
    states=[current_state]
    while current_state>0 and current_frame>1:
      
      current_frame-=1
      if current_state>2:
          to_check=[P[current_state][current_frame-1],P[current_state-1][current_frame-1],P[current_state-2][current_frame-1]]
          track=np.argmin(to_check)
      elif current_state>1:
          to_check=[P[current_state][current_frame-1],P[current_state-1][current_frame-1]]
          track=np.argmin(to_check)
      else:
          track=0
      if track==0:
          states.insert(0,current_state)
      elif track==1:
          current_state-=1
          states.insert(0,current_state)
      else:
          current_state-=2
          states.insert(0,current_state)
    return states
    

In [5]:
def seperate_templates(templates, num):
    states_info=[]
    for i in range(len(templates)):
        # For the ith template, the number of nodes in each state is initialized evenly
        nodes_num=len(templates[i])//num
        states_info.extend([[]])
        for state in range(1,num+1):
            for node in range(nodes_num):
                states_info[i].append(state)
        # assign the unassigned nodes to the last state
        unassigned_num=len(templates[i])-len(states_info[i])
        if unassigned_num>0:
            for m in range(unassigned_num):
                states_info[i].append(num)
    return states_info

In [6]:
def get_node_in_each_state(templates, state_num, states_info):
    node_in_each_state=[]
    for state in range(state_num+1):
        node_in_each_state.append([])
    for j in range(len(templates)):
        # print('len:',len(states_info[j]))
        for m in range(len(states_info[j])):
            k=int(states_info[j][m])
            node_in_each_state[k].append(templates[j][m])
    return node_in_each_state

In [7]:
def get_edge_cost(states_info,state_num):

    shift_prob=np.zeros((state_num+1,state_num+1))
    num_nodes_in_state=np.zeros(state_num+1)
    for i in range(len(states_info)):
        shift_prob[0][states_info[i][0]]+=1
    # count the state trainsition
    for i in range(len(states_info)):
        for j in range(len(states_info[i])-1):
            current_node=states_info[i][j]
            next_node=states_info[i][j+1]
            shift_prob[current_node][next_node]+=1
            num_nodes_in_state[current_node]+=1
        shift_prob[states_info[i][-2]][states_info[i][-1]]+=1
        num_nodes_in_state[states_info[i][-1]]+=1
    # get the probabilities of going from initial state to 1~state_num states 
    for j in range(state_num+1):
        N=len(states_info)
        N_0j=shift_prob[0][j]
        shift_prob[0][j]=N_0j/N
        if N_0j==0:
            shift_prob[0][j]=np.inf
        else:
            shift_prob[0][j]=-np.log(shift_prob[0][j])
    for i in range(1,state_num+1):
        for j in range(i,state_num+1):
            
            shift_prob[i][j]=shift_prob[i][j]/num_nodes_in_state[i]
            if shift_prob[i][j]==0:
                shift_prob[i][j]=np.inf
            else:
                shift_prob[i][j]=-np.log(shift_prob[i][j])
    return np.array(shift_prob)
        

In [8]:
from sklearn.cluster import KMeans
def Kmeans(nodes,Gaussian_num):

    if np.shape(nodes)[0]==0:
        mixture_model=GMMInfo()
        return mixture_model
    # initialize kmeans by using sklearn package   
    else:
        if np.shape(nodes)[0]>=4:
            kmeans=KMeans(n_clusters=Gaussian_num,random_state=0).fit(np.array(nodes))
        else:
            kmeans=KMeans(n_clusters=np.shape(nodes)[0],random_state=0).fit(np.array(nodes))
        mixture_model=GMMInfo()
        cov=[]
        mean=[]
        weight=np.zeros((Gaussian_num,1))
        for j in range(Gaussian_num):
            index=[]
            i=0
            w=0
            for k in kmeans.labels_:
                if j==k:
                    index.append(i)
                    w+=i
                i+=1
            weight[j]=w
            template=[nodes[m][:]for m in index]
            curr_mean=np.mean(template,axis=0)
            curr_cov=np.cov(np.array(template).T)
            diagonal_cov=np.diagonal(curr_cov,offset=0,axis1=0,axis2=1)
            cov.append(diagonal_cov)
            mean.append(curr_mean)
        #update the hmm model
        weight=weight/weight.sum()
        mixture_model.mean=mean
        mixture_model.var=cov
        mixture_model.weight=weight
        mixture_model.num=Gaussian_num

    return mixture_model
        
    

In [9]:
def Kmeans2(nodes_for_Kmeans,num_Gaussian_distribution):
        #initialize with mean, var and weight, with one cluster
        num_templates=len(nodes_for_Kmeans)
        means=[]
        covs=[]
        weights=[1]
        mean=np.mean(nodes_for_Kmeans,axis=0)
        cov=np.diagonal(np.cov(np.array(nodes_for_Kmeans).T),offset=0, axis1=0, axis2=1)
        means.append(mean)
        covs.append(cov)
        
        current_num_of_cluster=1
        episolom=0.04
        #initial should be 1 mean
        mix = GMMInfo()
        mix.var = np.array(covs)
        mix.mean = np.array(means)
        mix.num = current_num_of_cluster
        mix.weight = np.array(weights)
        stop=False
        
        while num_Gaussian_distribution>current_num_of_cluster and not stop:
            #now split
            new_means=[]
            new_covs=[]
            current_num_of_cluster=current_num_of_cluster*2
            new_clusters=[]
            for cluster in range(len(means)):
                #append newly two cluster center
                new_clusters.append([])
                new_clusters.append([])
                #get splitted mean and cov
                new_mean1=means[cluster]*(1-episolom)
                new_mean2=means[cluster]*(1+episolom)
                new_cov1=covs[cluster]*(1-episolom)
                new_cov2=covs[cluster]*(1+episolom)
                new_means.append(new_mean1)
                new_means.append(new_mean2)
                new_covs.append(new_cov1)
                new_covs.append(new_cov2)
            #now assign the templated into new clusters
            new_means=np.array(new_means)
            new_covs=np.array(new_covs)
            for node in nodes_for_Kmeans:
                d=log_Gaussian(new_means,new_covs,node)
                cluster=np.argmin(d)
                new_clusters[cluster].append(node)
            #now, according to the new clustered result, we get updated weight,
            #mean and cov
            means=[]
            covs=[]
            weights=[]
            #
            # print("For {} clusters, each cluster has following nodes".format(current_num_of_cluster))
            for cluster in new_clusters:
                # print(len(cluster))
                if len(cluster)<2*num_Gaussian_distribution:
                    stop=True
                    # print("For this state, we only have 2 Gaussian Distributions")
                mean=np.mean(cluster,axis=0)
                cov=np.cov(np.array(cluster).T)
                print(np.shape(cov))
                cov=np.diagonal(cov,offset=0, axis1=0, axis2=1)
                weight=len(cluster)/num_templates
                means.append(mean)
                covs.append(cov)
                weights.append(weight)
            #print(np.sum(weights))
            print("get {} means".format(current_num_of_cluster))
            # now, we put all the information to mix
            mix = GMMInfo()
            mix.var = np.array(covs)
            mix.mean = np.array(means)
            mix.num = current_num_of_cluster
            mix.weight = np.array(weights)
        return mix

In [10]:
def initialize_HMM(templates, state_num, Gaussian_num):
    hmm=HMMInfo()
    hmm.init=np.zeros((state_num,1))
    hmm.init[0]=1
    hmm.num=state_num

    states_info=seperate_templates(templates, state_num)
    node_in_each_state=get_node_in_each_state(templates, state_num, states_info)
    hmm.edge_cost=get_edge_cost(states_info,state_num)
    
    mix_models=[]
    for i in range(state_num):
        node_in_curr_state=node_in_each_state[i+1]
        curr_state_mix_model=Kmeans(node_in_curr_state, Gaussian_num[i])
        mix_models.append(curr_state_mix_model)
    hmm.mix=mix_models
    
    return hmm,states_info

In [11]:
# train hmm model
def trainhmm(templates,state_num,Gaussian_num):
    #initialize hmm model
    hmm,states_info=initialize_HMM(templates,state_num,Gaussian_num)
    best_dist=-np.inf
    curr_dist=0
    # use at most 99 iterations to train the model
    for i in range(1,100):
        for j in range(len(templates)):
            # use dtw to get the score and update alignment of the templates
            dist,states_info[j]=GMMHMM_DTW(hmm,templates[j])
            
            curr_dist+=dist
        hmm.edge_cost=get_edge_cost(states_info,state_num)
        # according to the alignment, classify the vectors of templates into different states
        node_in_each_state=get_node_in_each_state(templates,state_num,states_info)
        GMMs=[]
        for state in range(state_num):
            curr_state_node=node_in_each_state[state+1]
            # kmeans these vectors into 4 clusters to get weight, mean, var
            print(np.shape(curr_state_node))
            curr_mixture=Kmeans(curr_state_node,Gaussian_num[state])
            GMMs.append(curr_mixture)
        hmm.mix=GMMs
        # if it converges, break the iteration
        if abs(best_dist-curr_dist)<0.0015:
            break
        # update the best score
        best_dist=curr_dist
        # print(best_dist)
        curr_dist=0
    return hmm

In [12]:
# get MFCC of length 39
def getMFCC(wavename):
    import numpy as np
    import scipy.io.wavfile as wav
    from python_speech_features import mfcc
    fs, audio = wav.read(wavename)
    feature_mfcc = mfcc(audio, samplerate=fs)
    mfcc=[]
    mfcc.append(np.hstack([feature_mfcc[0],feature_mfcc[0],feature_mfcc[0]]))
    for i in range(1,len(feature_mfcc)-1):
        delta=np.zeros(13)
        for j in range(13):
            delta[j]=feature_mfcc[i+1][j]-feature_mfcc[i-1][j]
        mfcc.append(np.hstack([feature_mfcc[i],delta]))
    mfcc.append(np.hstack([feature_mfcc[-1],feature_mfcc[-1],feature_mfcc[-1]]))

    for i in range(1,len(mfcc)-1):
        acc=np.zeros(13)
        for j in range(13):
            acc[j]=mfcc[i+1][13+j]-mfcc[i-1][13+j]
        mfcc[i]=np.hstack([mfcc[i],acc])
    mfccs=np.array(mfcc)
    std=np.std(mfccs)
    var=np.var(mfccs,1)
    for i in range(len(mfccs)):
        for j in range(39):
            mfccs[i][j]=mfccs[i][j]/var[i]
    return mfccs

In [13]:
def GMMHMM(folder):
    models=[]
    for digit in range(10):
        templates=[]
        # for 2,6,8 we only use two states, since if we use more states there will be some states that have no nodes and thus cannot perofrom kmeans
        if digit in [5]:
            Gaussian_num=[1,1,1]
        # for other digits we use three states, the reason is similar to the previous annotation
        else: 
            Gaussian_num=[1,1,1]
        
        state_num=len(Gaussian_num)
        # 5 templates
        for i in range(1,11):
            mfcc=getMFCC(folder+'/'+str(digit)+'_training_'+str(i)+'.wav')
            templates.append(mfcc)
        # call trainhmm function to get the hmm model for each digit
        hmm=trainhmm(templates,state_num,Gaussian_num)
        models.append(hmm)
    
    return models

In [14]:
folder='digit_record'
gmmhmm_model=GMMHMM(folder)

C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks tha

(247, 39)
(422, 39)
(221, 39)
(258, 39)
(389, 39)
(243, 39)


C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks tha

(261, 39)
(373, 39)
(256, 39)
(263, 39)
(357, 39)


C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks tha

...

C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks tha

(307, 39)
(390, 39)
(293, 39)
(307, 39)
(390, 39)


C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks tha

(293, 39)
(307, 39)
(390, 39)
(293, 39)


C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\1\envs\compsci371\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks tha

In [15]:
hmm_models={}
for i in range(len(gmmhmm_model)):
    hmm_models[str(i)]=gmmhmm_model[i]

# Continuous Speech Recognition

## Node

In [16]:
class Node:
    def __init__(self,val, word):
        self.val=val
        self.next=[]
        self.word=word
        #three states: -1: root, 1: end, 0: others
        self.state=0        

## Lexical Tree

In [17]:
class LexicalTree:
    def __init__(self,models):
        self.getwords(models)
        zeros=np.zeros([39])
        ones=np.array([1 for i in range(39)])
        # set the initial state as a Gaussian mixture model
        initial_GMM=GMMInfo()
        initial_GMM.mean.append(zeros)
        initial_GMM.var.append(ones)
        initial_GMM.mean=np.array(initial_GMM.mean)
        initial_GMM.var=np.array(initial_GMM.var)
        initial_GMM.weight=[1]
        initial_GMM.num=1
        # let the initial state as the root of the lexical tree
        self.root=Node(initial_GMM,'*')
        self.root.state=-1
           
    # get model and transition cost for each digits
    def getwords(self,models):
        self.words=[]
        self.digits=list(models.keys())
        self.edge_cost={}
        for digit in self.digits:
            self.words.append(models[digit])
            self.edge_cost[digit]=models[digit].edge_cost

    def GenerateTree(self):
        for i in range(len(self.words)):
            word=self.words[i]
            digit=self.digits[i]
            # the initial state for ith digit
            previous=Node(word.mix[0],digit)
            # which is one of the next option for the root node
            self.root.next.append(previous)
            # other states for the digit
            for j in range(1,word.num):
                curr=Node(word.mix[j],digit)
                previous.next.append(curr)
                previous=curr
            # for the last state for the ith digit, its next node is root
            previous.next.append(self.root)
            previous.state=1 

In [18]:
lextree=LexicalTree(hmm_models)
lextree.GenerateTree()
root=lextree.root
edge_cost=lextree.edge_cost

## Continuous Speech Recognition

In [38]:
import copy
class ContinuousSpeechRecognition():
    def __init__(self):
        self.lextree=None
    # get the information from lextree    
    def get_tree_info(self,root,edge_cost):
        self.lextree=root
        self.get_nodes(self.lextree)
        self.edge_cost=edge_cost
        self.previous={}
        self.next={}
        self.end_nodes=[]
        for i in range(len(self.nodes)):
            node=self.nodes[i]
            # if the state of the node is 1, it is the end of a word(digit)
            if node.state==1:
                self.end_nodes.append(i)
            self.next[i]=[]
            if len(node.next)>0:
                for next_node in node.next:
                    # get the previous node(s) and next node(s) for each node
                    self.next[i].append(self.nodes.index(next_node))
                    self.previous[self.nodes.index(next_node)]=i

    def get_nodes(self,root):
        self.nodes=[root]
        self.init_nodes=[]
        self.states=[0]
        for node in root.next:
            state=0
            curr_node=node
            # add the first node of each word into init_nodes list
            self.init_nodes.append(node)
            # while the node is not the ending node of a word(digit)
            while curr_node.state!=1:
                state+=1
                self.nodes.append(curr_node)
                self.states.append(state)
                curr_node=curr_node.next[0]
            # add the ending node of a word to the nodes list
            state+=1
            self.nodes.append(curr_node)
            self.states.append(state)

    
    # continuous speech recognition for 4 or 7 digits phone number
    def digit_recognition_47(self,data,loop_cost=-5):
        zeros=np.zeros(np.shape(data)[1])
        data=np.vstack([zeros,data])
        cols=len(data)
        rows=len(self.nodes)
        # cost matrix
        costs=np.full([rows,cols],np.inf)
        # the cost from the initial state '*' to other nodes
        init_cost=copy.deepcopy(costs)
        init_cost[0][0]=0
        
        all_costs=[init_cost]
        for i in range(1,cols):
            for j in range(len(all_costs)):
                curr_costs=all_costs[j]
                for k in range(1,rows):
                    # calculate the log gaussian score of the vector in input data and the node
                    score=mixture_log_Gaussian(self.nodes[k].val,data[i])
                    cost=min(curr_costs[k][i-1]+self.edge_cost[self.nodes[k].word][self.states[k]][self.states[k]], #horizontal transition from itself
                             curr_costs[self.previous[k]][i-1]+self.edge_cost[self.nodes[k].word][self.states[self.previous[k]]][self.states[k]]) # diagonal transition from its previous node
                    # cost for this node = current best path score + edge cost + node cost (log Gaussian score)
                    curr_costs[k][i]=cost+score
                # find the index of the node with the minimum cost
                min_index=np.argmin(curr_costs[:,i])
                min_cost=min(curr_costs[:,i])
                # when the minimum cost is at one of the end nodes
                if min_index in self.end_nodes:
                    #if the jth costs matrix is in the middle of the all costs, we've already get the word(digit)
                    if len(all_costs)-1>j:
                        next_costs=all_costs[j]
                        next_costs[0,i]=min_cost+loop_cost
                    # if the j is the last costs matrix of the all costs
                    # create a new costs matrix for the alignment of a new digit, if the number of words (digits) is less than 7
                    elif len(all_costs)<7:
                        new_costs=copy.deepcopy(costs)
                        new_costs[0,i]=min_cost+loop_cost
                        all_costs.append(new_costs)
        # according to the all_costs matrix to get the words(digits)
        result=self.get_words_47(all_costs,i)
        return result
        
        
    def get_words_47(self,all_costs,i):
        if len(all_costs)>=7:
            # for 7 digits the costs matrix of the last digit is the 7th matrix
            min_cost_7=min(all_costs[6][self.end_nodes,i])
            # for 4 digits the costs matrix of the last digit is the 4th matrix
            min_cost_4=min(all_costs[3][self.end_nodes,i])
            
            # judge whether the data is more likely to be 4 digits or 7 digits by their minimum costs
            if min_cost_7<min_cost_4:
                end_index=6
            else:
                end_index=3
        else:
            end_index=3
            
        result=''
        # from the last word (digit) to the first
        for j in range(end_index,-1,-1):
            curr_word,i=self.get_curr_word(all_costs[j],i)
            result=curr_word+result
        return result

    def get_curr_word(self,curr_costs,i):
        # In the last col of the costs matrix, get the end node with minimum cost
        min_index=np.argmin(curr_costs[self.end_nodes,i])
        curr_node=self.end_nodes[min_index]
        
        while i>0 and curr_node>0:
            min_previous_cost=min(curr_costs[curr_node][i-1],curr_costs[self.previous[curr_node]][i-1])
            # horizontal move
            if min_previous_cost==curr_costs[curr_node][i-1]:
                i-=1
            # diagonal move
            elif min_previous_cost==curr_costs[self.previous[curr_node]][i-1]:
                i-=1
                curr_node=self.previous[curr_node]
                
        return self.nodes[self.end_nodes[min_index]].word,i

    def digit_recognition(self,data,loop_cost=0):
        zeros=np.zeros(np.shape(data)[1])
        data=np.vstack([zeros,data])
        cols=len(data)
        rows=len(self.nodes)
        # cost matrix
        costs=np.full([rows,cols],np.inf)
        costs[0][0]=0

        for i in range(1,cols):
            for j in range(1,rows):
                # calculate the log gaussian score of the vector in input data and the node
                score=mixture_log_Gaussian(self.nodes[j].val,data[i])
                cost=min(costs[j][i-1]+self.edge_cost[self.nodes[j].word][self.states[j]][self.states[j]],#horizontal transition from itself
                      costs[self.previous[j]][i-1]+self.edge_cost[self.nodes[j].word][self.states[self.previous[j]]][self.states[j]]) # diagonal transition from its previous node
                if not cost==np.inf:
                    costs[j][i]=cost+score
            # find the index of the node with the minimum cost
            min_index=np.argmin(costs[:,i])
            min_cost=min(costs[:,i])
            # when the minimum cost is at one of the end nodes
            if min_index in self.end_nodes and min_cost!=np.inf:
                # add a new word (digit)
                costs[0,i]=min_cost+loop_cost
        # according to the costs matrix to get the words(digits)
        result=self.get_words(costs,i)
        return result

    def get_words(self,costs,i):
        result=''
        while i>0:
            curr_word,i=self.get_curr_word(costs,i)
            result=curr_word+result
        return result
        

In [39]:
csr=ContinuousSpeechRecognition()
csr.get_tree_info(root,edge_cost)

## Problem 1 Testing

In [28]:
import os 
foldername="test_data/Problem_1/"
folder=os.listdir(foldername)
for filename in folder:
    phone_num=filename[:-4]
    data=getMFCC(foldername+filename)
    result=csr.digit_recognition_47(data)
    
    print("The test phone num is {}, the recongnition result is {}".format(phone_num,result))

The test phone num is 0145, the recongnition result is 0095
The test phone num is 0355, the recongnition result is 3995
The test phone num is 1538, the recongnition result is 5885
The test phone num is 2096401, the recongnition result is 0955
The test phone num is 2964, the recongnition result is 2564955
The test phone num is 3608952, the recongnition result is 9822
The test phone num is 5332016, the recongnition result is 9382160
The test phone num is 9115672, the recongnition result is 9626


## Problem 2 Testing

In [42]:
foldername="test_data/Problem_2/"
folder=os.listdir(foldername)
for filename in folder:
    phone_num=filename[:-4]
    data=getMFCC(foldername+filename)
    result=csr.digit_recognition(data)

    print("The test phone num is {}, the recongnition result is {}".format(phone_num,result))

The test phone num is 123456, the recongnition result is 720569
The test phone num is 2212, the recongnition result is 3052
The test phone num is 37472941, the recongnition result is 5372709521
The test phone num is 55555, the recongnition result is 59995
The test phone num is 911385, the recongnition result is 79593905
